In [1]:
from functools import partial
from pathlib import Path

import bw2calc as bc
import bw2data as bd
import bw2io as bi
from project_path import ROOT_DIR

In [2]:
# bd.projects

In [3]:
bd.projects.set_current("default")

In [4]:
# bd.databases

## import Land-system Chang method
 > Global

In [5]:
method_category = "LandSystemChange_Global"

In [6]:
SIMAPRO_DIR = ROOT_DIR + r"\data\external\from_simapro"

In [7]:
STORE_DATA_TO = Path(
    ROOT_DIR + r"\src\aesa_pbs\data" + f"\\aesa_{method_category}.xlsx"
)

In [8]:
si = bi.SimaProLCIACSVImporter(
    filepath=Path(SIMAPRO_DIR + f"\PBsLCIAv072_{method_category}.csv")
)

Extracted 1 methods in 0.01 seconds


In [9]:
# si.data[0].keys()

In [10]:
for method in si.data:
    print(method["name"], method["unit"])

('PBs-LCIA (baseline)', 'Land-system change - Global') %


In [11]:
si.data[0]["exchanges"][0]#.keys()

{'amount': 1.56e-12,
 'CAS number': '',
 'categories': ('Raw', 'land'),
 'name': 'Transformation, from forest',
 'unit': 'm2'}

In [12]:
si.apply_strategies()
si.statistics()

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: link_iterable_by_fields
Applying strategy: match_subcategories
Applied 10 strategies in 0.41 seconds
1 methods
22 cfs
22 unlinked cfs


(1, 22, 22)

In [13]:
link_by_name_and_categories = partial(
    bi.strategies.link_iterable_by_fields,
    other=bd.Database(bd.config.biosphere),
    kind="biosphere",
    fields=("name", "categories"),
)

In [14]:
si.apply_strategy(link_by_name_and_categories)
si.statistics()

Applying strategy: link_iterable_by_fields
1 methods
22 cfs
4 unlinked cfs


(1, 22, 4)

In [15]:
# list(si.unlinked)

In [16]:
# from the 5 unlinked:
# 1 has amount 0, can be dropped - Carbon dioxide, in air
# 2 others "Carbon dioxide" and "Carbon monoxide" are not in biosphere3, BUT
# "Carbon dioxide, fossil" 
# and "Carbon monoxide, fossil" and "Carbon monoxide, non-fossil" (with same CFs)
# have been linked (see check_equivalent_linked)
[(exc["name"], exc["categories"]) for exc in list(si.unlinked)]

[('Transformation, from forest, natural', ('natural resource', 'land')),
 ('Transformation, from forest, secondary', ('natural resource', 'land')),
 ('Transformation, to forest, natural', ('natural resource', 'land')),
 ('Transformation, to forest, used', ('natural resource', 'land'))]

In [17]:
def see_unlinked(imported_methods):
    unlinked_exc_names = [
        exc_unlinked["name"] for exc_unlinked in list(imported_methods.unlinked)
    ]
    for ix in range(len(imported_methods.data)):
        print(imported_methods.data[ix]["name"][1])
        for exc in imported_methods.data[ix]["exchanges"]:
            if exc["name"] in unlinked_exc_names:
                print("\t",
                    exc["name"],
                    exc["categories"],
                    exc["amount"],
                )
        print("\n")
        # if exc["amount"] != 0

In [18]:
def check_equivalent_linked(imported_methods, list_names):
    for ix in range(len(imported_methods.data)):
        print(imported_methods.data[ix]["name"][1])
        for exc in imported_methods.data[ix]["exchanges"]:
            for x in list_names:
                if x in exc["name"]:
                    print("\t",
                        exc["name"],
                        exc["categories"],
                        exc["amount"],
                    )
        print("\n")

In [19]:
def remove_duplicates_if_any(imported_methods):
    for ix in range(len(imported_methods.data)):
        current_exc = len(imported_methods.data[ix]["exchanges"])
        wo_duplicates_exc = len(
            {tuple(o["input"]) for o in imported_methods.data[ix]["exchanges"]}
        )
        print(
            imported_methods.data[ix]["name"][1], ": ", current_exc, wo_duplicates_exc,
        )
        if current_exc != wo_duplicates_exc:
            unique_exchanges = {
                tuple(o["input"]): o for o in imported_methods.data[ix]["exchanges"]
            }
            imported_methods.data[ix]["exchanges"] = list(unique_exchanges.values())
            print("Duplicates removed.\n")
        else:
            print("No duplicates.\n")

In [20]:
see_unlinked(si)

Land-system change - Global
	 Transformation, from forest, natural ('natural resource', 'land') 1.56e-12
	 Transformation, from forest, secondary ('natural resource', 'land') 1.56e-12
	 Transformation, to forest, natural ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, used ('natural resource', 'land') -1.56e-12




In [21]:
b3 = bd.Database("biosphere3")
[
    flow
    for flow in b3
    if "transformation, to forest" in flow["name"].lower()
]

['Transformation, to forest, unspecified' (square meter, None, ('natural resource', 'land')),
 'Transformation, to forest, primary (non-use)' (square meter, None, ('natural resource', 'land')),
 'Transformation, to forest, extensive' (square meter, None, ('natural resource', 'land')),
 'Transformation, to forest, intensive' (square meter, None, ('natural resource', 'land')),
 'Transformation, to forest, secondary (non-use)' (square meter, None, ('natural resource', 'land'))]

In [22]:
check_equivalent_linked(si, ["Transformation, to forest"])

Land-system change - Global
	 Transformation, to forest, extensive ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, intensive ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, intensive ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, intensive ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, intensive ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, natural ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, primary (non-use) ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, secondary (non-use) ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, unspecified ('natural resource', 'land') -1.56e-12
	 Transformation, to forest, used ('natural resource', 'land') -1.56e-12




In [23]:
si.drop_unlinked()
si.statistics()

Applying strategy: drop_unlinked_cfs
Applied 1 strategies in 0.00 seconds
1 methods
18 cfs
0 unlinked cfs


(1, 18, 0)

In [24]:
# check if there are duplicates (both numbers should be equal)
remove_duplicates_if_any(si)

Land-system change - Global :  18 10
Duplicates removed.



In [25]:
si.statistics()

1 methods
10 cfs
0 unlinked cfs


(1, 10, 0)

In [26]:
# Next si.write_excel(method_category) will create an excel file in 
# ~\Local\pylca\Brightway3\default###\output\lcia-matching-{method_category}.xlsx

# Go to created excel and apply manual corrections
#TODO: manual corrections should not be manual

# Store corrected excel to STORE_DATA_TO